In [ ]:
%pip install wordcloud
%pip install seaborn
%pip install ipywidgets
%pip install nltk

## ETL

In [ ]:
#Extraction
projects = pd.read_csv('projects.csv')
tags = pd.read_csv('tags.csv')
projects.head()

In [ ]:
tags.head()

In [ ]:
#Transform
df = pd.merge(tags,projects,on='id')
df.head()

In [ ]:
df.tag.isnull()

In [ ]:
df = df.loc[df.tag.notnull(),:]

In [ ]:
#Save the transformed data
df.to_csv('labeled_projects.csv',index=False)

## EDA

In [ ]:
from collections import Counter
import ipywidgets as widgets
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from wordcloud import WordCloud, STOPWORDS
sns.set_theme()
warnings.filterwarnings("ignore")
import re

In [2]:
df = pd.read_csv('labeled_projects.csv')

In [3]:
df.head()

,id,tag,created_on,title,description
0,6,computer-vision,2020-02-20 06:43:18,Comparison between YOLO and RCNN on real world...,Bringing theory to experiment is cool. We can ...
1,7,computer-vision,2020-02-20 06:47:21,"Show, Infer & Tell: Contextual Inference for C...",The beauty of the work lies in the way it arch...
2,9,graph-learning,2020-02-24 16:24:45,Awesome Graph Classification,"A collection of important graph embedding, cla..."
3,15,reinforcement-learning,2020-02-28 23:55:26,Awesome Monte Carlo Tree Search,A curated list of Monte Carlo tree search pape...
4,19,graph-learning,2020-03-03 13:54:31,Diffusion to Vector,Reference implementation of Diffusion2Vec (Com...


In [ ]:
#Tags distribution
df.tag.hist()

In [ ]:
#Wordcloud of description feature
def get_wc(df,tag):
    tag_titles = df.loc[df.tag==tag,'title']
    cloud = WordCloud(stopwords=STOPWORDS,collocations=False,width=500, height=300).generate(' '.join(tag_titles.to_list()))
    plt.imshow(cloud)


In [ ]:
get_wc(df,'natural-language-processing')

In [ ]:
get_wc(df,'mlops')

In [ ]:
get_wc(df,'time-series')

In [ ]:
df.tag.unique()

## Preprocessing

In [4]:
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
# nltk.download('wordnet')
# nltk.download('omw-1.4')

In [20]:
df = pd.read_csv('labeled_projects.csv')

In [21]:
df.head()

,id,tag,created_on,title,description
0,6,computer-vision,2020-02-20 06:43:18,Comparison between YOLO and RCNN on real world...,Bringing theory to experiment is cool. We can ...
1,7,computer-vision,2020-02-20 06:47:21,"Show, Infer & Tell: Contextual Inference for C...",The beauty of the work lies in the way it arch...
2,9,graph-learning,2020-02-24 16:24:45,Awesome Graph Classification,"A collection of important graph embedding, cla..."
3,15,reinforcement-learning,2020-02-28 23:55:26,Awesome Monte Carlo Tree Search,A curated list of Monte Carlo tree search pape...
4,19,graph-learning,2020-03-03 13:54:31,Diffusion to Vector,Reference implementation of Diffusion2Vec (Com...


In [22]:
df['text'] = df.title +' ' + df.description
df.text 

0      Comparison between YOLO and RCNN on real world...
1      Show, Infer & Tell: Contextual Inference for C...
2      Awesome Graph Classification A collection of i...
3      Awesome Monte Carlo Tree Search A curated list...
4      Diffusion to Vector Reference implementation o...
                             ...                        
950    A Survey of the State of Explainable AI for NL...
951    Topic Modeling with BERT Leveraging 🤗 Transfor...
952    OpenMMLab Computer Vision MMCV is a python lib...
953    Machine Learning Methods Explained (+ Examples...
954    Rasoee A powerful web and mobile application t...
Name: text, Length: 955, dtype: object

In [23]:
# Clean the text
def clean_text(text,stopwords):

    text = text.lower()
    
    #Remove non-alphanumeric
    text = re.sub(r"([!\"'#$%&()*\+,-./:;<=>?@\\\[\]^_`{|}~])", r" \1 ", text )  
    text = re.sub("[^A-Za-z0-9]+", " ", text) 

    #Remove links
    text = re.sub('http\S+',"",text)
    
    #tokenize
    text = text.split()

    #Remove  stopwords 
    new_text = []
    for word in text:   
        if word not in stopwords:
            new_text.append(word)
    text = new_text 
    
    # Stem
    stemmer = PorterStemmer()
    text = stemmer.stem(' '.join(text))
    
    return text

In [24]:
# 2. lower all the text tokens
df['text'] = df['text'].apply(clean_text,args=(STOPWORDS,))

In [25]:
df.head()

,id,tag,created_on,title,description,text
0,6,computer-vision,2020-02-20 06:43:18,Comparison between YOLO and RCNN on real world...,Bringing theory to experiment is cool. We can ...,comparison yolo rcnn real world videos bringin...
1,7,computer-vision,2020-02-20 06:47:21,"Show, Infer & Tell: Contextual Inference for C...",The beauty of the work lies in the way it arch...,show infer tell contextual inference creative ...
2,9,graph-learning,2020-02-24 16:24:45,Awesome Graph Classification,"A collection of important graph embedding, cla...",awesome graph classification collection import...
3,15,reinforcement-learning,2020-02-28 23:55:26,Awesome Monte Carlo Tree Search,A curated list of Monte Carlo tree search pape...,awesome monte carlo tree search curated list m...
4,19,graph-learning,2020-03-03 13:54:31,Diffusion to Vector,Reference implementation of Diffusion2Vec (Com...,diffusion vector reference implementation diff...


In [35]:
# Remove other irrelevant tags
def remove_oos_labels(df,label,default,freq):
    oos_labels = []
    for tag,tag_freq in Counter(df[label]).most_common():
        if tag_freq<freq:
            oos_labels.append(tag)
    df.loc[df[label].isin(oos_labels), label] = default
    return df



In [38]:
# df = remove_oos_labels(df,'tag','other',75)
Counter(df.tag)

Counter({'computer-vision': 356,
         'other': 132,
         'natural-language-processing': 388,
         'mlops': 79})

In [39]:
#Label Ecnoding
from sklearn.preprocessing import LabelEncoder


In [40]:
le = LabelEncoder()
df['tag'] = le.fit_transform(df['tag'])

In [41]:
df.head()

,id,tag,created_on,title,description,text
0,6,0,2020-02-20 06:43:18,Comparison between YOLO and RCNN on real world...,Bringing theory to experiment is cool. We can ...,comparison yolo rcnn real world videos bringin...
1,7,0,2020-02-20 06:47:21,"Show, Infer & Tell: Contextual Inference for C...",The beauty of the work lies in the way it arch...,show infer tell contextual inference creative ...
2,9,3,2020-02-24 16:24:45,Awesome Graph Classification,"A collection of important graph embedding, cla...",awesome graph classification collection import...
3,15,3,2020-02-28 23:55:26,Awesome Monte Carlo Tree Search,A curated list of Monte Carlo tree search pape...,awesome monte carlo tree search curated list m...
4,19,3,2020-03-03 13:54:31,Diffusion to Vector,Reference implementation of Diffusion2Vec (Com...,diffusion vector reference implementation diff...


In [42]:
le.classes_

array(['computer-vision', 'mlops', 'natural-language-processing', 'other'],
      dtype=object)

In [44]:
le.transform(['computer-vision', 'mlops',
       'natural-language-processing', 'other'])

array([0, 1, 2, 3])

In [45]:
df.head()

,id,tag,created_on,title,description,text
0,6,0,2020-02-20 06:43:18,Comparison between YOLO and RCNN on real world...,Bringing theory to experiment is cool. We can ...,comparison yolo rcnn real world videos bringin...
1,7,0,2020-02-20 06:47:21,"Show, Infer & Tell: Contextual Inference for C...",The beauty of the work lies in the way it arch...,show infer tell contextual inference creative ...
2,9,3,2020-02-24 16:24:45,Awesome Graph Classification,"A collection of important graph embedding, cla...",awesome graph classification collection import...
3,15,3,2020-02-28 23:55:26,Awesome Monte Carlo Tree Search,A curated list of Monte Carlo tree search pape...,awesome monte carlo tree search curated list m...
4,19,3,2020-03-03 13:54:31,Diffusion to Vector,Reference implementation of Diffusion2Vec (Com...,diffusion vector reference implementation diff...


In [47]:
df.drop(['created_on','title','description','id'],inplace=True,axis=1)

In [48]:
df.head()

,tag,text
0,0,comparison yolo rcnn real world videos bringin...
1,0,show infer tell contextual inference creative ...
2,3,awesome graph classification collection import...
3,3,awesome monte carlo tree search curated list m...
4,3,diffusion vector reference implementation diff...


In [49]:
df.to_csv('prep_labelled_projects.csv',index=False)


## Splitting

In [56]:
# Split using test_train split of sklearn
from sklearn.model_selection import train_test_split
import pandas as pd


In [57]:
df = pd.read_csv('prep_labelled_projects.csv')
df.head()

,tag,text
0,0,comparison yolo rcnn real world videos bringin...
1,0,show infer tell contextual inference creative ...
2,3,awesome graph classification collection import...
3,3,awesome monte carlo tree search curated list m...
4,3,diffusion vector reference implementation diff...


In [58]:
X_train,X_,y_train,y_ = train_test_split(df.text.to_numpy(),df.tag.to_numpy(),test_size=0.3,stratify=df.tag)
X_train

array(['torch points3d pytorch framework deep learning point cloud',
       'semantic graphs generating deep questions deep question generation dqg aims generate complex questions require reasoning multiple pieces information input passag',
       'delight deep light weight transformers similar better performance transformer based models significantly fewer paramet',
       'machine learning graphs model comprehensive taxonomy propose simple framework graphedm comprehensive taxonomy review unify several graph representation learning method',
       'upside reinforcement learning implementation udrl outlined juergen schmidhuber arxiv org abs 1912 02875',
       'serving pytorch models production amazon sagemaker torchserve now natively supported amazon sagemaker default model server pytorch infer',
       'gentle introduction text summarization machine learning text summarization technique generating concise precise summary voluminous texts focusing sections convey useful info',
       

In [59]:
X_val,X_test,y_val,y_test = train_test_split(X_,y_,test_size=0.5,stratify=y_)

In [60]:
print("X_train, X_val, X_test: ",len(X_train),len(X_val), len(X_test))
print("y_train, y_val, y_test: ",len(y_train),len(y_val), len(y_test))


X_train, X_val, X_test:  668 143 144
y_train, y_val, y_test:  668 143 144


In [69]:
train_df = pd.DataFrame(X_train,columns = ['text'])
train_df.head()

,text
0,torch points3d pytorch framework deep learning...
1,semantic graphs generating deep questions deep...
2,delight deep light weight transformers similar...
3,machine learning graphs model comprehensive ta...
4,upside reinforcement learning implementation u...


## Augmentation

In [61]:
# %pip install nlpaug==1.1.0 
# %pip install transformers==3.0.2 
# %pip install snorkel==0.9.8 


In [62]:
import re
from snorkel.augmentation import transformation_function
import random

In [63]:
# Aliases
aliases_by_tag = {
    "computer vision": ["cv", "vision"],
    "mlops": ["production"],
    "natural language processing": ["nlp", "nlproc"]
}


In [64]:
res = aliases_by_tag.copy()
for tag,alias in aliases_by_tag.items():
    for idx in range(len(alias)):
        res[alias[idx]] = [tag] + alias[idx+1:] + alias[:idx]
print(res)
aliases_by_tag = res

{'computer vision': ['cv', 'vision'], 'mlops': ['production'], 'natural language processing': ['nlp', 'nlproc'], 'cv': ['computer vision', 'vision'], 'vision': ['computer vision', 'cv'], 'production': ['mlops'], 'nlp': ['natural language processing', 'nlproc'], 'nlproc': ['natural language processing', 'nlp']}


In [65]:
# \b matches spaces
def find_word(word, text):
    word = word.replace("+", "\+")
    pattern = re.compile(fr"\b({word})\b", flags=re.IGNORECASE)
    return pattern.search(text)


In [76]:
#Define a swap aliases function to replace the aliases in a given text feature.find_word
#with random words from the matched list
@transformation_function()
def swap_aliases(x):
    matches = []
    for tag,aliases in aliases_by_tag.items():
        match = find_word(tag,x.text)
        if match:
            matches.append( (tag,match) )
    if len(matches):
        tag,match = random.choice( matches )
        x.text = x.text.replace( tag, random.choice(aliases_by_tag[tag]) )
    return x


In [77]:
# Define augmentation policy with certain ruules
from snorkel.augmentation import ApplyOnePolicy, PandasTFApplier
policy = ApplyOnePolicy(5,keep_original=True)
tf_applier = PandasTFApplier([swap_aliases],policy=policy)
train_df_aug = tf_applier.apply(train_df)

100%|██████████| 668/668 [00:01<00:00, 607.50it/s]


In [84]:
train_df_aug.drop_duplicates(subset=["text"], inplace=True)


In [85]:
print("Original train df: ",len(train_df) )
print("Augmented train df: ",len(train_df_aug))


Original train df:  668
Augmented train df:  913


In [87]:
train_df_aug.head(10)

,text
0,torch points3d pytorch framework deep learning...
1,semantic graphs generating deep questions deep...
2,delight deep light weight transformers similar...
3,machine learning graphs model comprehensive ta...
4,upside reinforcement learning implementation u...
5,serving pytorch models production amazon sagem...
5,serving pytorch models mlops amazon sagemaker ...
6,gentle introduction text summarization machine...
7,image synthesis cvpr 2020 overview different a...
8,intellicode compose code generation using tran...
